In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from collections import Counter
import re

In [ ]:
df = pd.read_csv('postings.csv')
df.head()

In [ ]:
#First let's see some basic review of the data 
df.describe()

In [ ]:
# Now let's see some basic info of the data that we have 
df.info()

In [ ]:
# The primary objective of the data cleaning of this project is to clean the data so that any person who uses this dataset after 
# it has been cleaned , it would be easier for that person to know about the data engineer jobs 
# First data cleaning project , Here are the 20 data cleaning steps we are gonna take to make sure the data is tidy to Explore 
# 1. Drop rows with missing essential values
# 2. Drop duplicates
# 3. Convert to datetime
# 4. Extract year
# 5. Fill missing categorical values
# 6. Fill missing categorical values
# 7. Strip whitespaces
# 8. Convert to lowercase
# 9. Standardize country names
# 10. Handle 'remote' locations
# 11. Function to standardize text
# 12. Apply text standardization
# 13. Split skills into lists
# 14. Explode list columns
# 15. Aggregate exploded data
# 16. Reset index
# 17. Rename aggregated column
# 18. Drop original list column
# 19. Merge aggregated data
# 20. Display cleaned data

In [ ]:
# 1. Drop rows with missing essential values
df.dropna(subset=['job_title', 'company', 'job_location'], inplace=True)

In [ ]:
# 2. Drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# 3. Convert to datetime
df['first_seen'] = pd.to_datetime(df['first_seen'])

In [ ]:
# 4. Extract year
df['year'] = df['first_seen'].dt.year

In [ ]:
# 5. Fill missing categorical values
df['job level'] = df['job level'].fillna('Not Specified')

In [ ]:
# 6. Fill missing categorical values
df['job_type'] = df['job_type'].fillna('Not Specified')

In [ ]:
# 7. Strip whitespaces
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
# 8. Convert to lowercase
df['job_title'] = df['job_title'].str.lower()

In [ ]:
# 9. Standardize country names
df['search_country'] = df['search_country'].replace('united states of america', 'usa')

In [ ]:
# 10. Handle 'remote' locations
df['job_location'] = df['job_location'].replace('remote', 'not specified')

In [ ]:
# 11. Function to standardize text
def standardize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text
    else:
        return text

In [ ]:
# 12. Apply text standardization
df['job_summary'] = df['job_summary'].astype(str).apply(standardize_text)

In [ ]:
# 13. Split skills into lists
df['job_skills'] = df['job_skills'].str.split(', ')

In [ ]:
# 14. Explode list columns
df_exploded = df.explode('job_skills')

In [ ]:
# 15. Aggregate exploded data
df_agg = df_exploded.groupby([col for col in df_exploded.columns if col != 'job_skills'], as_index=False)['job_skills'].agg(list)

In [ ]:
# 16. Reset index
df_agg = df_agg.reset_index(drop=True)

In [ ]:
# 17. Rename aggregated column
df_agg = df_agg.rename(columns={'job_skills': 'job_skills_agg'})

In [ ]:
# 18. Drop original list column
df = df.drop('job_skills', axis=1)

In [ ]:
# 19. Merge aggregated data
df_cleaned = df.merge(df_agg, on=[col for col in df.columns if col != 'job_skills'], how='left')

In [ ]:
# 20. Display cleaned data
print(df_cleaned.head().to_markdown(index=False, numalign="left", stralign="left"))

In [ ]:
### Now that we have completed data cleaning , we can now simply move on to EDA and here are some things we are gonna do with the code
# Here are what we are gonna do in the EDA phase of this project and what are the results of all the visualization , what each visualization 
# shows and what findings did we find from the project
# Calculate the number of job postings per year & Display the results in a table
# Visualize the distribution of job postings over time using a line plot
# Identify the top 10 companies with the most job postings & Display the results in a table
# Visualize the top 10 companies using a bar plot
# Identify the top 10 job locations with the most job postings & Display the results in a table
# Visualize the top 10 job locations using a bar plot
# Show the distribution of `job_type` using a pie chart
# Show the distribution of `job level` using a pie chart
# This is what we are going to do in the dataset, 
# Skill Analysis
# Calculate skill frequency and get top 20
# Bar plot of top 20 skills
# This bar chart illustrates the top 20 most sought-after skills in the dataset, providing insights into the key qualifications employers are seeking.
# Job Title Analysis
# Get top 20 job titles
# Bar plot of top 20 job titles
# This bar chart displays the 20 most common job titles, shedding light on the prevalent roles in the job market.
# Company and Job Title Analysis
# Get top 20 company-job title combinations
# Bar plot of top 20 company-job title combinations
# This bar chart reveals the top 20 combinations of companies and job titles, indicating which companies are hiring for specific roles.
# Job Location Analysis (Excluding 'Not Specified')
# Filter out 'not specified' locations
# Get top 20 job locations
# Bar plot of top 20 job locations (excluding 'not specified')
# This bar chart presents the 20 most common job locations, excluding those that are not specified, offering insights into geographic hiring trends.
# Job Location Analysis (USA Only)
# Filter for USA jobs
# Get top 20 job locations in USA
# Bar plot of top 20 job locations in USA
# This bar chart focuses on the 20 most frequent job locations within the USA, providing a more targeted view of hiring trends in the country.
# Number of Skills per Job Analysis
# Calculate number of skills per job
# Histogram and box plot of number of skills
# The histogram and box plot illustrate the distribution of the number of skills required per job, revealing the range and central tendency of skill requirements.


In [ ]:
# Calculate the number of job postings per year
job_postings_per_year = df_cleaned['year'].value_counts().reset_index()
job_postings_per_year.columns = ['Year', 'Number of Job Postings']


In [ ]:

# Display the results in a table
print("Number of Job Postings Per Year:")
print(job_postings_per_year.to_markdown(index=False, numalign="left", stralign="left"))



In [ ]:
# Visualize the distribution of job postings over time using a line plot
plt.figure(figsize=(10, 6))
plt.plot(job_postings_per_year['Year'], job_postings_per_year['Number of Job Postings'], marker='o', linestyle='-')
plt.title('Job Postings Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Job Postings')
plt.grid(axis='y', linestyle='--')
plt.xticks(job_postings_per_year['Year'], rotation=45)
plt.show()


In [ ]:

# Identify the top 10 companies with the most job postings
top_companies = df_cleaned['company'].value_counts().head(10).reset_index()
top_companies.columns = ['Company', 'Number of Job Postings']


In [ ]:

# Display the results in a table
print("\nTop 10 Companies with the Most Job Postings:")
print(top_companies.to_markdown(index=False, numalign="left", stralign="left"))



In [ ]:
# Visualize the top 10 companies using a bar plot
plt.figure(figsize=(12, 6))
plt.bar(top_companies['Company'], top_companies['Number of Job Postings'])
plt.title('Top 10 Companies by Number of Job Postings')
plt.xlabel('Company')
plt.ylabel('Number of Job Postings')
plt.xticks(rotation=45, ha='right')
plt.show()



In [ ]:
# Identify the top 10 job locations with the most job postings
top_locations = df_cleaned['job_location'].value_counts().head(10).reset_index()
top_locations.columns = ['Job Location', 'Number of Job Postings']



In [ ]:
# Display the results in a table
print("\nTop 10 Job Locations with the Most Job Postings:")
print(top_locations.to_markdown(index=False, numalign="left", stralign="left"))


In [ ]:

# Visualize the top 10 job locations using a bar plot
plt.figure(figsize=(12, 6))
plt.bar(top_locations['Job Location'], top_locations['Number of Job Postings'])
plt.title('Top 10 Job Locations by Number of Job Postings')
plt.xlabel('Job Location')
plt.ylabel('Number of Job Postings')
plt.xticks(rotation=45, ha='right')
plt.show()



In [ ]:
# Show the distribution of `job_type` using a pie chart
job_type_counts = df_cleaned['job_type'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(job_type_counts, labels=job_type_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Job Types')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()


In [ ]:

# Show the distribution of `job level` using a pie chart
job_level_counts = df_cleaned['job level'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(job_level_counts, labels=job_level_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Job Levels')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()


In [ ]:


# Skill Analysis

# Calculate skill frequency and get top 20
all_skills = [skill for sublist in df_cleaned['job_skills_agg'] for skill in sublist if isinstance(sublist, list)]
skill_counts = Counter(all_skills)
top_20_skills = skill_counts.most_common(20)


In [ ]:

# Bar plot of top 20 skills
plt.figure(figsize=(12, 6))
plt.bar([skill[0] for skill in top_20_skills], [skill[1] for skill in top_20_skills])
plt.title('Top 20 Most Frequent Skills')
plt.xlabel('Skill')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()
#This bar chart illustrates the top 20 most sought-after skills in the dataset, providing insights into the key qualifications employers are seeking.



In [ ]:
# Job Title Analysis

# Get top 20 job titles
top_20_job_titles = df_cleaned['job_title'].value_counts().head(20)

# Bar plot of top 20 job titles
plt.figure(figsize=(12, 6))
plt.bar(top_20_job_titles.index, top_20_job_titles.values)
plt.title('Top 20 Most Frequent Job Titles')
plt.xlabel('Job Title')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()
#This bar chart displays the 20 most common job titles, shedding light on the prevalent roles in the job market.



In [ ]:
# Company and Job Title Analysis

# Get top 20 company-job title combinations
top_20_company_job_title = df_cleaned.groupby(['company', 'job_title']).size().reset_index(name='counts').nlargest(20, 'counts')

# Bar plot of top 20 company-job title combinations
plt.figure(figsize=(12, 6))
plt.bar(top_20_company_job_title.apply(lambda x: f"{x['company']} - {x['job_title']}", axis=1), top_20_company_job_title['counts'])
plt.title('Top 20 Company-Job Title Combinations')
plt.xlabel('Company - Job Title')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()
#This bar chart reveals the top 20 combinations of companies and job titles, indicating which companies are hiring for specific roles.



In [ ]:
# Job Location Analysis (Excluding 'Not Specified')

# Filter out 'not specified' locations
df_filtered = df_cleaned[df_cleaned['job_location'] != 'not specified']

# Get top 20 job locations
top_20_job_locations = df_filtered['job_location'].value_counts().head(20)

# Bar plot of top 20 job locations (excluding 'not specified')
plt.figure(figsize=(12, 6))
plt.bar(top_20_job_locations.index, top_20_job_locations.values)
plt.title('Top 20 Most Frequent Job Locations (excluding "not specified")')
plt.xlabel('Job Location')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()
#This bar chart presents the 20 most common job locations, excluding those that are not specified, offering insights into geographic hiring trends.



In [ ]:
# Job Location Analysis (USA Only)

# Filter for USA jobs
df_usa = df_cleaned[df_cleaned['search_country'] == 'usa']

# Get top 20 job locations in USA
top_20_usa_job_locations = df_usa['job_location'].value_counts().head(20)

# Bar plot of top 20 job locations in USA
plt.figure(figsize=(12, 6))
plt.bar(top_20_usa_job_locations.index, top_20_usa_job_locations.values)
plt.title('Top 20 Most Frequent Job Locations in USA')
plt.xlabel('Job Location')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()
#This bar chart focuses on the 20 most frequent job locations within the USA, providing a more targeted view of hiring trends in the country.



In [ ]:
# Number of Skills per Job Analysis

# Calculate number of skills per job
df_cleaned['num_skills'] = df_cleaned['job_skills_agg'].apply(len)

# Histogram and box plot of number of skills
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6), sharey=True)
axes[0].hist(df_cleaned['num_skills'], bins=10, edgecolor='black')
axes[0].set_title('Distribution of Number of Skills per Job')
axes[0].set_xlabel('Number of Skills')
axes[0].set_ylabel('Frequency')
axes[1].boxplot(df_cleaned['num_skills'])
axes[1].set_title('Box Plot of Number of Skills per Job')
axes[1].set_xlabel('Number of Skills')
plt.tight_layout()
plt.show()
#The histogram and box plot illustrate the distribution of the number of skills required per job, revealing the range and central tendency of skill requirements.
